In [1]:
import pandas as pd
import matplotlib as mpl
from scipy import stats
import scipy.special
from scipy import constants
from colossus.cosmology import cosmology
from astropy.cosmology import z_at_value
params = {'flat': True, 'H0': 67.77, 'Om0': 0.307, 'Ob0': 0.04825, 'sigma8': 0.8288, 'ns': 0.9611}
cosmology.addCosmology('planck14', params)
cosmo = cosmology.setCosmology('planck14')

In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
import matplotlib as mpl
from matplotlib import gridspec
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator, FixedLocator
from matplotlib.backends.backend_pdf import PdfPages

plt.style.use("shao.mplstyle")
fontSize = 15
lineWidth = 1.5

colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', \
          u'#bcbd22', u'#17becf']

In [35]:
def read_data(file, mass_min = 100, r_min = 300):
# within 300kpc, > mass_min, after tidal destruction
    h = 0.6776999831199646
    with h5py.File(file, 'r' ) as hf:
        # data = hf[inputFile]
        # print ("data.keys = " , data.keys())
        # gid = data['GalaxyID'][:]
        # num = len(gid)
        print("hf.keys()", hf.keys())
        # print("hf/satellite_galaxies.keys()",hf['satellite_galaxies'].keys())
        t = hf["header_info/time"][:]
        a = hf["header_info/scale_factor"][:]
        central_Mhalo = hf['central_galaxy/Mhalo'][:]
        satellite_mstar = hf['satellite_galaxies/Mstar'][:]
        # satellite_msub = hf['satellite_galaxies/Msub'][:]
        satellite_M200 = hf['satellite_galaxies/M200'][:]
        satellite_pos = hf['satellite_galaxies/pos'][:]
        # central_pos = hf['satellite_galaxies/pos'][:]
        satellite_vel = hf['satellite_galaxies/vel'][:]
        # is_MW_central = hf['satellite_galaxies/is_MW_central'][:]
        # Last snapshot at which the galaxy is a FOF central subhalo. Different from Galform definition of central galaxy.
        # satellite_snap_last_FOF_central = hf['satellite_galaxies/snap_last_FOF_central'][:]
        # Last snapshot at which the galaxy was a central.
        # satellite_snap_last_central = hf["satellite_galaxies/snap_last_central"][:]
        # satellite_type = hf["satellite_galaxies/type"][:]
        # destruction = hf["tidal_disruption/destroyed_orbit_interpolation"][:]
    print("d<300,m>0 satellite num=", len(satellite_mstar))
    # print("d<300,m>100 satellite num=", sum(satellite_mstar[:,0]>100))
    dis = []
    for i in range(len(satellite_pos)):
        dis_ = (satellite_pos[i,0,0]**2 + satellite_pos[i,0,1]**2 + satellite_pos[i,0,2]**2)**0.5
        dis.append(dis_)
    # R200 = 227.2 * (central_Mhalo/1.252e12)**0.3333
    # is_satellite = np.logical_and(satellite_mstar[:,0]>mass_min, destruction==False)
    # satellite_mstar = satellite_mstar[is_satellite]
    # satellite_pos = satellite_pos[is_satellite]
    # satellite_mhalo = satellite_mhalo[is_satellite]
    # satellite_vel = satellite_vel[is_satellite]
    # print("d<300,m>100 after destruction satellite num=", sum(is_satellite))
    satellite_data =pd.DataFrame({
            "satellite_mstar":satellite_mstar[:,0],
            "satellite_M200":satellite_M200, 
            "distance[kpc]":dis},
        columns=['satellite_mstar', 'satellite_M200', 'distance[kpc]'])
    satellite_data = satellite_data.sort_values(by=['satellite_M200'],ascending=False)
    return satellite_data, satellite_pos, satellite_vel, t, a

In [41]:
def read_hydro(file_num, simulation, maxn):
    # read the subgroup catalogue
    # inputFile_root = "groups_199_z000p000/eagle_subfind_tab_028_z000p000.%i.hdf5" 
    file = simulation_name
    inputFile_root = '/home/zhaox/Magpie/' + file + '/groups_199_z000p000/eagle_subfind_tab_199_z000p000.%i.hdf5'
    # read the data - read the first tab file to get the total number of subgroups
    inputFile = inputFile_root % 0

    # 对于4和5，中央星系的galaxyID不等于0

    if simulation == 'zcut7':
        if file_num == 4:
            centerIndex = 2
        elif file_num == 5:
            centerIndex = 2
        else:
            centerIndex = 1
    elif simulation == '7DM_GAS':
        if file_num == 2:
            centerIndex = 2
        elif file_num == 4:
            centerIndex = 3
        else:
            centerIndex = 1
    else:
        print("can't understand simulation_name!")
        return
    with h5py.File( inputFile, 'r' ) as hf:
        header  = hf['Header'].attrs
        # print ('header keys:', header.keys())
        numFiles = header['NTask']  # this gives the number of files in which the group catalog is saved
        noTotGrps = header['TotNgroups']  # total number of FOF groups
        noTotSubs = header['TotNsubgroups']  # total number of SUBFIND subhaloes
        hFactor   = header['HubbleParam']
        # print("All groups in the file:", hf.keys())
        # print("All entries in the 'FOF' group:", hf['FOF'].keys())
        # print("All entries in the 'SubHalo' group:", hf['Subhalo'].keys())
        # print(numFiles, noTotGrps, noTotSubs, hFactor)


    # reserve memory for the output arrays
    FOF_length   = np.zeros( noTotGrps, np.uint32 )
    FOF_mass     = np.zeros( noTotGrps, np.float32 )
    FOF_pos      = np.zeros( (noTotGrps,3), np.float32 )  # centre of potential
    FOF_M200     = np.zeros( noTotGrps, np.float32 )
    FOF_R200     = np.zeros( noTotGrps, np.float32 )
    Subhalo_mass = np.zeros( noTotSubs, np.float32 )
    Subhalo_pos  = np.zeros( (noTotSubs,3), np.float32 )  # centre of potential
    Subhalo_vel = np.zeros( (noTotSubs,3), np.float32 )
    Subhalo_gnum = np.zeros( noTotSubs, np.float32 )

    # loop over all the various files
    start, end = 0, 0
    start2, end2 = 0, 0
    for i in range(numFiles):
        inFile = inputFile_root % i
        with h5py.File( inFile, 'r' ) as hf:
            h = hf['Header'].attrs
            numGrps = h['Ngroups']  # number of groups this file
            numsubGrps = h['Nsubgroups']
            end = start + numGrps
            end2 = start2 + numsubGrps
    #         print(numsubGrps)
            # print ("Reading file %i of %i --%s-- which contains %i groups ..." % ( i+1, numFiles, inFile, numGrps ))
            
            # FOF_length[start:end] = hf["FOF/GroupLength"]
            # FOF_mass[start:end] = hf["FOF/GroupMass"]
            FOF_pos[start:end]  = hf["FOF/GroupCentreOfPotential"]
            FOF_M200[start:end] = hf["FOF/Group_M_Crit200"]
            FOF_R200[start:end] = hf["FOF/Group_R_Crit200"]
            Subhalo_pos[start2:end2] = hf["Subhalo/CentreOfPotential"]
            Subhalo_mass[start2:end2] = hf["Subhalo/MassType"][:,4]
            Subhalo_vel[start2:end2] = hf["Subhalo/Velocity"]
            Subhalo_gnum[start2:end2] = hf["Subhalo/GroupNumber"]
            start = end
            start2 = end2
    Subhalo_pos = (Subhalo_pos - FOF_pos[centerIndex-1])* 1000/hFactor
    satellite_index = []
    dis = []
    for i in range(len(Subhalo_pos)):
        dis_ = ((Subhalo_pos[i][0]**2 + Subhalo_pos[i][1]**2 + Subhalo_pos[i][2]**2)**0.5)
        # 筛选条件小于300kpc，大于100msun
        # print(dis_)
        if  dis_  < 300 and Subhalo_mass[i] > 0:
            satellite_index.append(i)
            dis.append(dis_)
    # print(satellite_index)
    # 选择groupNumber==centerindex的星系
    # satellite_index = np.where(Subhalo_gnum == centerIndex)

    # print(satellite_index)
    print('r200 = ',FOF_R200[centerIndex-1])
    #计算相对坐标和相对速度
    # satellite_pos = (Subhalo_pos[satellite_index] - FOF_pos[centerIndex-1])* 1000/hFactor
    satellite_pos = Subhalo_pos[satellite_index]
    satellite_vel = Subhalo_vel[satellite_index] - Subhalo_vel[satellite_index][0]
    dis = np.linalg.norm(satellite_pos, axis = 1)
    satellite_data =pd.DataFrame({"groupNumber":Subhalo_gnum[satellite_index], "id":satellite_index,\
            "massType4":Subhalo_mass[satellite_index]*1e10/hFactor, "distance[kpc]":dis, \
            "x[kpc]":satellite_pos[:,0],"y[kpc]":satellite_pos[:,1],"z[kpc]":satellite_pos[:,2],\
            "vx[km/s]":satellite_vel[:,0],"vy[km/s]":satellite_vel[:,1],"vz[km/s]":satellite_vel[:,2]},  \
        columns=['groupNumber', 'id', 'mass', 'distance[kpc]', 'x[kpc]', 'y[kpc]', 'z[kpc]', 'vx[km/s]', 'vy[km/s]', 'vz[km/s]'])
        # 返回显示质量最大的maxn个,maxn=1000表示输出所有满足条件的星系，从1开始是因为0是中央星系
    # satellite_data = all_data[all_data['groupNumber'] == centerIndex+1] 
    # satellite_data = satellite_data.query("mass > 0 & `distance[kpc]` < 300")
    # satellite_index = np.where(satellite_data['groupNumber'] == centerIndex)
    # satellite_data = satellite_data.iloc[satellite_index]
    satellite_data = satellite_data.sort_values(by=['massType4'],ascending=False)
    # satellite_data = satellite_data.loc[satellite_data['distance[kpc]']<300 ]
    satellite_pos = np.array(satellite_data[['x[kpc]','y[kpc]','z[kpc]']])
    satellite_vel = np.array(satellite_data[['vx[km/s]','vy[km/s]','vz[km/s]']])

    print('center pos:', FOF_pos[centerIndex-1], "satellite num = ", len(satellite_pos)-1)
    return FOF_M200[centerIndex-1], satellite_data.iloc[:maxn+1], satellite_pos[1:maxn+1], satellite_vel[1:maxn+1]
    # return satellite_data.sort_values(by=['mass'],ascending=False).iloc[:]


def pos_match(hydro_pos, galform_pos):
    match = np.full(len(hydro_pos), -1,dtype=int)
    for i in range(len(hydro_pos)):
        for j in range(len(galform_pos)):
            if (np.linalg.norm(hydro_pos[i] - galform_pos[j])) < 1:
                match[i] = j
                continue
    return match

In [17]:
simulation_num = 2
file_num = simulation_num
simulation = 'zcut7'
r = 500
h = 0.6777
simulation_name = 'M' + str(simulation_num) + '_' + simulation
file = "/home/zhaox/app/别人的代码/Destruction/satellite_orbits_Galform/M" + str(simulation_num) + "_MR.hdf5"

In [36]:
satellite_data, MR_gal_pos, MR_gal_vel, t, a = read_data(file)
 

hf.keys() <KeysViewHDF5 ['central_galaxy', 'header_info', 'metadata', 'satellite_galaxies']>
d<300,m>0 satellite num= 141


In [38]:
satellite_data[:10]

,satellite_mstar,satellite_M200,distance[kpc]
4,2.012489e+09,1.943852e+11,150.785834
7,3.351127e+08,9.785745e+10,151.340566
3,2.400457e+08,6.094287e+10,239.529229
25,9.474621e+07,5.027348e+10,161.137292
14,4.428966e+07,4.131924e+10,14.471185
35,3.766205e+07,4.051442e+10,23.890576
39,3.608204e+07,3.940486e+10,60.291129
42,3.074200e+07,2.101023e+10,43.281451
135,5.485639e+06,2.037891e+10,271.605474
27,7.035872e+06,1.872268e+10,86.204104


In [39]:
MR_hydro_M200, MR_hydro_data, MR_hydro_pos, MR_hydro_vel =  read_hydro(1, simulation, maxn=1000)

r200 =  0.1473333
center pos: [26.079184 31.32167  42.181503] satellite num =  30


In [40]:
MR_hydro_data[:10]

,groupNumber,id,mass,distance[kpc],x[kpc],y[kpc],z[kpc],vx[km/s],vy[km/s],vz[km/s]
0,1.0,0,1.653749e+10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.0,1,5.461249e+09,150.785843,95.783974,-113.047768,-27.964315,-58.044437,-34.037460,-43.726036
2,1.0,2,4.558275e+08,239.529236,-148.982590,-126.391052,-138.577591,55.552925,-54.533829,-106.620247
4,1.0,6,4.044446e+08,161.137299,98.283195,-121.440445,-39.469761,-96.926697,-51.774094,-3.997475
21,1.0,64,7.104646e+06,86.204109,-2.513299,46.109035,72.792778,-39.511097,41.250336,41.620262
20,1.0,62,5.943091e+06,89.128426,42.180073,-68.756866,37.910561,-39.441837,-84.366318,-124.098274
22,1.0,73,2.373382e+06,106.244873,-34.369991,-100.399658,-5.156061,-43.416859,-84.063950,19.662807
3,1.0,3,1.466248e+06,271.605499,-270.817047,13.987786,-15.231771,112.692085,23.389694,-156.922516
8,1.0,17,1.420080e+06,158.659897,155.227844,22.726635,-23.680731,-134.447449,137.111771,-10.692341
6,1.0,13,1.281717e+06,174.302948,147.071579,-26.188400,-89.808907,-12.361385,-126.803970,-134.401031


In [9]:
MR_galform_satellite_data[:15]

,id,mass,distance[kpc],x[kpc],y[kpc],z[kpc],vx[km/s],vy[km/s],vz[km/s]
0,0,3.551462e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,6,2.012489e+09,150.783541,95.781250,-113.046875,-27.964844,-58.044437,-34.037460,-43.726036
9,10,3.351127e+08,151.336166,95.406250,-114.019531,-28.281250,-38.856976,8.293182,-59.538246
4,5,2.400457e+08,239.530438,-148.984375,-126.390625,-138.578125,55.552925,-54.533829,-106.620247
155,256,1.282878e+08,484.962143,313.226562,369.804688,-17.941406,-101.367065,-31.662384,-74.449287
29,32,9.474621e+07,161.133649,98.281250,-121.437500,-39.468750,-96.926697,-51.774094,-3.997475
17,19,4.428966e+07,14.473809,-7.800781,-3.636719,-11.636719,-118.724579,-17.497345,-97.872887
40,44,3.766205e+07,23.890370,-17.546875,-13.035156,-9.640625,98.265961,86.473450,110.957817
44,48,3.608204e+07,60.288879,26.402344,54.179688,-1.492188,15.544701,89.041611,-24.427460
47,52,3.074200e+07,43.282593,31.113281,29.875000,-3.582031,18.554035,14.189072,53.095703


In [8]:
pos_match(HR_galform_pos, MR_galform_pos)

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1])

In [10]:
HR_galform_satellite_data[:15]

,id,mass,distance[kpc],x[kpc],y[kpc],z[kpc],vx[km/s],vy[km/s],vz[km/s]
0,387,1.565343e+10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117,512,5.540344e+08,329.316728,325.179688,17.203125,49.109375,97.169586,-87.958099,-98.567276
4,391,2.489460e+08,144.399740,68.414062,-119.515625,-43.437500,-80.370247,-14.568420,-39.833172
135,588,1.179640e+08,417.792744,236.597656,337.812500,-66.746094,-117.542549,-59.816010,-85.739441
33,424,4.415096e+07,30.711287,23.199219,15.917969,12.312500,-67.366402,-93.836212,-57.029236
30,421,4.037430e+07,36.741209,26.468750,24.355469,7.492188,105.647789,42.321548,-46.565556
34,425,3.977454e+07,22.496650,20.699219,-4.250000,-7.718750,-136.050018,4.585007,-39.690426
15,405,3.398129e+07,44.447536,-41.187500,-16.269531,-3.804688,-61.436363,-36.283279,15.501781
21,412,1.551137e+07,108.892177,-53.410156,60.863281,72.804688,25.502243,-46.441910,11.893307
23,414,7.453495e+06,136.151006,60.320312,-97.800781,-73.031250,-92.732018,-50.625214,0.153985


In [12]:
simulation = '7DM_GAS'
file = "/home/zhaox/Magpie/Destruction/satellite_orbits_Galform/M" + str(simulation_num) + "_7DM_GAS.hdf5"

    
satellite_msub, satellite_mstar, satellite_pos, satellite_vel, central_pos,central_Mhalo = read_data(file, after_destruction=False)
sort = np.argsort(satellite_mstar[:,0])[::-1] # 大到小
satellite_pos = satellite_pos[sort]
satellite_vel = satellite_vel[sort]
satellite_mstar = satellite_mstar[sort]
satellite_msub = satellite_msub[sort]
print("central_Mhalo=%e"%central_Mhalo[0])
print("satellite_msub=",satellite_msub[:,0])

NameError: name 'read_data' is not defined